In [1]:
import tensorflow as tf
import numpy as np
import os

/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## use .np file

In [2]:
sess = tf.Session()
vgg_path = '../vgg_model'
tf.saved_model.loader.load(sess, ["vgg16"], vgg_path)
graph = tf.get_default_graph()
image_input = graph.get_tensor_by_name('image_input:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')
layer3 = graph.get_tensor_by_name('layer3_out:0')
layer4 = graph.get_tensor_by_name('layer4_out:0')
layer7 = graph.get_tensor_by_name('layer7_out:0')

INFO:tensorflow:Restoring parameters from ../vgg_model/variables/variables


In [3]:
[n.name for n in tf.get_default_graph().as_graph_def().node ]

['image_input',
 'keep_prob',
 'Processing/split/split_dim',
 'Processing/split',
 'Processing/sub/y',
 'Processing/sub',
 'Processing/sub_1/y',
 'Processing/sub_1',
 'Processing/sub_2/y',
 'Processing/sub_2',
 'Processing/concat/axis',
 'Processing/concat',
 'conv1_1/filter/Initializer/Const',
 'conv1_1/filter',
 'conv1_1/filter/Assign',
 'conv1_1/filter/read',
 'conv1_1/L2Loss',
 'conv1_1/weight_loss/y',
 'conv1_1/weight_loss',
 'conv1_1/Conv2D',
 'conv1_1/biases/Initializer/Const',
 'conv1_1/biases',
 'conv1_1/biases/Assign',
 'conv1_1/biases/read',
 'conv1_1/BiasAdd',
 'conv1_1/Relu',
 'conv1_2/filter/Initializer/Const',
 'conv1_2/filter',
 'conv1_2/filter/Assign',
 'conv1_2/filter/read',
 'conv1_2/L2Loss',
 'conv1_2/weight_loss/y',
 'conv1_2/weight_loss',
 'conv1_2/Conv2D',
 'conv1_2/biases/Initializer/Const',
 'conv1_2/biases',
 'conv1_2/biases/Assign',
 'conv1_2/biases/read',
 'conv1_2/BiasAdd',
 'conv1_2/Relu',
 'pool1',
 'conv2_1/filter/Initializer/Const',
 'conv2_1/filter',
 

In [4]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
reuse_vars

[<tf.Variable 'conv1_1/filter:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv1_2/filter:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_2/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2_1/filter:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv2_1/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2_2/filter:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'conv2_2/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv3_1/filter:0' shape=(3, 3, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_1/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3_2/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_2/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3_3/filter:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_3/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 

## use .npz file

In [5]:
import numpy as np
weights = np.load("../vgg/vgg16_weights.npz")

In [6]:

class airbus_model():
    def __init__(self, image_shape=(768, 768), model_dir="./save_model", learning_rate=0.0001, continue_training=False,
                 vgg_path="../vgg/vgg16_weights.npz"):
        self.image_shape = image_shape
        self.model_dir = model_dir
        if not os.path.isdir(self.model_dir):
            os.makedirs(self.model_dir)

        self.lr = learning_rate
        self.continue_training = continue_training
        self.vgg_path = vgg_path

        self.sess = tf.Session()

        self.x_holder = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1], 3],
                                       name="x_holder")
        self.y_holder = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1]],
                                       name="y_holder")
        self.final = self.conv_layers()
        self.out = tf.sigmoid(self.final, name="final_out")
        reshaped_logits = tf.reshape(self.final, (-1, self.image_shape[0] * self.image_shape[1]))
        reshaped_labels = tf.reshape(self.y_holder, (-1, self.image_shape[0] * self.image_shape[1]))
        self.loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=reshaped_logits, labels=reshaped_labels),
            name="cross_entropy")
        self.train_op = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss, name="train_op")
        
        self.sess.run(tf.global_variables_initializer())
        
        #print(model.sess.run("vgg_conv1_1/kernel:0"))
        for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
            print(i.name)

    def conv_layers(self):
        ############   down ##############
        # build vgg
        conv1_1 = tf.layers.conv2d(self.x_holder, filters=64, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv1_1')
        conv1_2 = tf.layers.conv2d(conv1_1, filters=64, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv1_2')
        pool1 = tf.layers.max_pooling2d(conv1_2, pool_size=(2, 2), strides=2, padding='same', name='vgg_pool1')

        conv2_1 = tf.layers.conv2d(pool1, filters=128, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv2_1')
        conv2_2 = tf.layers.conv2d(conv2_1, filters=128, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv2_2')
        pool2 = tf.layers.max_pooling2d(conv2_2, pool_size=(2, 2), strides=2, padding='same', name='vgg_pool2')

        conv3_1 = tf.layers.conv2d(pool2, filters=256, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv3_1')
        conv3_2 = tf.layers.conv2d(conv3_1, filters=256, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv3_2')
        conv3_3 = tf.layers.conv2d(conv3_2, filters=256, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv3_3')
        pool3 = tf.layers.max_pooling2d(conv3_3, pool_size=(2, 2), strides=2, padding='same', name='vgg_pool3')

        conv4_1 = tf.layers.conv2d(pool3, filters=512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv4_1')
        conv4_2 = tf.layers.conv2d(conv4_1, filters=512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv4_2')
        conv4_3 = tf.layers.conv2d(conv4_2, filters=512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv4_3')
        pool4 = tf.layers.max_pooling2d(conv4_3, pool_size=(2, 2), strides=2, padding='same', name='vgg_pool4')

        conv5_1 = tf.layers.conv2d(pool4, filters=512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv5_1')
        conv5_2 = tf.layers.conv2d(conv5_1, filters=512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv5_2')
        conv5_3 = tf.layers.conv2d(conv5_2, filters=512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu,
                                   trainable=False, name='vgg_conv5_3')
        pool5 = tf.layers.max_pooling2d(conv5_3, pool_size=(2, 2), strides=2, padding='same', name='vgg_pool5')

        #############  up  ###################
        # https://datascience.stackexchange.com/questions/26451/how-to-calculate-the-output-shape-of-conv2d-transpose
        layer6_1 = tf.layers.conv2d_transpose(pool5, filters=512, kernel_size=(3, 3), strides=(2, 2), padding='same',
                                              bias_initializer=tf.constant_initializer(0.1),
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                              activation=tf.nn.relu, trainable=True, name='new_layer6_1')
        layer6_2 = tf.concat([pool4, layer6_1], axis=-1, name='new_layer6_2')

        layer7_1 = tf.layers.conv2d_transpose(layer6_2, filters=256, kernel_size=(3, 3), strides=(2, 2), padding='same',
                                              bias_initializer=tf.constant_initializer(0.1),
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                              activation=tf.nn.relu, trainable=True, name='new_layer7_1')
        layer7_2 = tf.concat([pool3, layer7_1], axis=-1, name='new_layer7_2')

        layer8_1 = tf.layers.conv2d_transpose(layer7_2, filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same',
                                              bias_initializer=tf.constant_initializer(0.1),
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                              activation=tf.nn.relu, trainable=True, name='new_layer8_1')
        layer8_2 = tf.concat([pool2, layer8_1], axis=-1, name='new_layer8_2')

        layer9_1 = tf.layers.conv2d_transpose(layer8_2, filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same',
                                              bias_initializer=tf.constant_initializer(0.1),
                                              kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                              activation=tf.nn.relu, trainable=True, name='new_layer9_1')
        layer9_2 = tf.concat([pool1, layer9_1], axis=-1, name='new_layer9_2')

        # the last layer should be linear, no activation function
        logits = tf.layers.conv2d_transpose(layer9_2, filters=1, kernel_size=(3, 3), strides=(2, 2), padding='same',
                                            bias_initializer=tf.constant_initializer(0.1),
                                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                            activation=None, trainable=True, name='new_logits')
        return tf.squeeze(logits, axis=-1)  # remove the channel dimension which has size 1

    def load_weights(self):
        weights = np.load(self.vgg_path)
        d = {var.name: var for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) if var.name.startswith('vgg_')}
        #print(d.keys())
        for new_name in d.keys():
            # observe d.keys() and weights.keys() to see the difference in naming
            if new_name.endswith('/kernel:0'):
                old_name = new_name[4:11] + '_W'
            elif new_name.endswith('/bias:0'):
                old_name = new_name[4:11] + '_b'
            self.sess.run(d[new_name].assign(weights[old_name]))
            # print(old_name)

tf.reset_default_graph()
model = airbus_model()

vgg0 = model.sess.run("vgg_conv1_1/kernel:0")
new0 = model.sess.run("new_layer6_1/kernel:0")
model.load_weights()
vgg1 = model.sess.run("vgg_conv1_1/kernel:0")
new1 = model.sess.run("new_layer6_1/kernel:0")



vgg_conv1_1/kernel:0
vgg_conv1_1/bias:0
vgg_conv1_2/kernel:0
vgg_conv1_2/bias:0
vgg_conv2_1/kernel:0
vgg_conv2_1/bias:0
vgg_conv2_2/kernel:0
vgg_conv2_2/bias:0
vgg_conv3_1/kernel:0
vgg_conv3_1/bias:0
vgg_conv3_2/kernel:0
vgg_conv3_2/bias:0
vgg_conv3_3/kernel:0
vgg_conv3_3/bias:0
vgg_conv4_1/kernel:0
vgg_conv4_1/bias:0
vgg_conv4_2/kernel:0
vgg_conv4_2/bias:0
vgg_conv4_3/kernel:0
vgg_conv4_3/bias:0
vgg_conv5_1/kernel:0
vgg_conv5_1/bias:0
vgg_conv5_2/kernel:0
vgg_conv5_2/bias:0
vgg_conv5_3/kernel:0
vgg_conv5_3/bias:0
new_layer6_1/kernel:0
new_layer6_1/bias:0
new_layer7_1/kernel:0
new_layer7_1/bias:0
new_layer8_1/kernel:0
new_layer8_1/bias:0
new_layer9_1/kernel:0
new_layer9_1/bias:0
new_logits/kernel:0
new_logits/bias:0
beta1_power:0
beta2_power:0
new_layer6_1/kernel/Adam:0
new_layer6_1/kernel/Adam_1:0
new_layer6_1/bias/Adam:0
new_layer6_1/bias/Adam_1:0
new_layer7_1/kernel/Adam:0
new_layer7_1/kernel/Adam_1:0
new_layer7_1/bias/Adam:0
new_layer7_1/bias/Adam_1:0
new_layer8_1/kernel/Adam:0
new

In [7]:
# none of them are the same, expect: False
(vgg0 == vgg1).any()

False

In [8]:
# all of them are the same, expect: True
(new0 == new1).all()

True

## use ckpt

If we have access to the code of the graph, then first run that code. Then we can load the status of the model with the saved checkpoint as follows:

In [9]:
#tf.reset_default_graph()

# build the graph here

sess = tf.Session()
restore_saver = tf.train.Saver()
restore_saver.restore(sess, os.path.join("../vgg/save_model", "airbus_model"))
# if in the original directory, we can also resore by
# restore_saver.restore(sess, tf.train.latest_checkpoint("../vgg/save_model"))

INFO:tensorflow:Restoring parameters from ../vgg/save_model/airbus_model


If we don't have access to the code, we can first restore the graph structure and then load the variable values:

In [10]:

sess = tf.Session()
tf.reset_default_graph()
saver = tf.train.import_meta_graph(os.path.join("../vgg/save_model", "airbus_model.meta"))
saver.restore(sess, os.path.join("../vgg/save_model", "airbus_model"))

INFO:tensorflow:Restoring parameters from ../vgg/save_model/airbus_model


In [11]:
sess.graph.get_operations()

[<tf.Operation 'x_holder' type=Placeholder>,
 <tf.Operation 'y_holder' type=Placeholder>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform/shape' type=Const>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform/min' type=Const>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform/max' type=Const>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform/sub' type=Sub>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform/mul' type=Mul>,
 <tf.Operation 'vgg_conv1_1/kernel/Initializer/random_uniform' type=Add>,
 <tf.Operation 'vgg_conv1_1/kernel' type=VariableV2>,
 <tf.Operation 'vgg_conv1_1/kernel/Assign' type=Assign>,
 <tf.Operation 'vgg_conv1_1/kernel/read' type=Identity>,
 <tf.Operation 'vgg_conv1_1/bias/Initializer/zeros' type=Const>,
 <tf.Operation 'vgg_conv1_1/bias' type=VariableV2>,
 <tf.Operation 'vgg_conv1_1/bias/Assign' type=Assign>,


In [12]:
#print("train_op:")
[n.name for n in sess.graph.get_operations() if "train_op" in n.name ]

#print("placeholders:")
#[n.name for n in sess.graph.get_operations() if n.type=="Placeholder" ]

['train_op/learning_rate',
 'train_op/beta1',
 'train_op/beta2',
 'train_op/epsilon',
 'train_op/update_new_layer6_1/kernel/ApplyAdam',
 'train_op/update_new_layer6_1/bias/ApplyAdam',
 'train_op/update_new_layer7_1/kernel/ApplyAdam',
 'train_op/update_new_layer7_1/bias/ApplyAdam',
 'train_op/update_new_layer8_1/kernel/ApplyAdam',
 'train_op/update_new_layer8_1/bias/ApplyAdam',
 'train_op/update_new_layer9_1/kernel/ApplyAdam',
 'train_op/update_new_layer9_1/bias/ApplyAdam',
 'train_op/update_new_logits/kernel/ApplyAdam',
 'train_op/update_new_logits/bias/ApplyAdam',
 'train_op/mul',
 'train_op/Assign',
 'train_op/mul_1',
 'train_op/Assign_1',
 'train_op']

Then we can retrieve the tensor in the graph for our tasks, similar to the case using .np file:

In [13]:
graph = tf.get_default_graph()
x = graph.get_tensor_by_name("x_holder:0")
y = graph.get_tensor_by_name("y_holder:0")
train_op = graph.get_operation_by_name("train_op")